In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
from PIL import Image
from utils import read_mat_file, visualize_sample_dataset, vae_loss
from dataloader import SVHDDataset, SVHDDigitOnlyDigitDataset
import scipy.io

from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from model import VGG
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
import os
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
# Train the model
from model import VGG16, VAE
import cv2 
import torch
import torch.nn as nn
import torch.nn.functional as F

os.environ['TORCH_NNPACK'] = '0'

In [2]:
train_path = '/data/omscs_datasets/train/'
train_mat_file_path = train_path + 'digitStruct.mat'
test_path = '/data/omscs_datasets/train/'
test_mat_file_path = test_path + 'digitStruct.mat'
device = torch.device("cuda")

In [3]:
train_dataset = SVHDDigitOnlyDigitDataset(train_mat_file_path, train_path,mode='train')

## train ##


In [4]:
train_images = []
train_labels = []
for t in tqdm(train_dataset):
    for tt in t:
        train_images.append(tt['image'])
        train_labels.append(tt['label'])

33402it [03:01, 183.60it/s]          


In [5]:
train_images[0].shape

torch.Size([3, 32, 32])

In [6]:
# 이미지 텐서를 쌓습니다
stacked_images = torch.stack(train_images, dim=0)

# 채널별 평균과 표준편차를 계산합니다
mean = torch.mean(stacked_images, dim=[0, 2, 3])
std = torch.std(stacked_images, dim=[0, 2, 3])

print(f"Calculated mean: {mean}")
print(f"Calculated std: {std}")

Calculated mean: tensor([-2.9629e-05, -2.1441e-04, -2.2989e-04])
Calculated std: tensor([1.0002, 0.9999, 1.0001])


In [9]:
train_transform = transforms.Compose([
    #transforms.Resize((32, 32)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.4342, 0.4431, 0.4768], std=[0.1927, 0.1956, 0.1931])  # 일반적인 ImageNet 평균 및 표준편차

])

from PIL import Image
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        sample = {'image': image, 'label': label}
        return sample

In [10]:
print(len(train_images), len(train_labels))
train_dataset = CustomDataset(train_images, train_labels, transform=train_transform)

73257 73257


In [11]:
print(len(train_images), len(train_labels))
train_dataset = CustomDataset(train_images, train_labels, transform=train_transform)

73257 73257


In [12]:
# Specify the lengths of the training and testing sets
train_size = int(1 * len(train_dataset))
test_size = len(train_dataset) - train_size

# Use random_split to split the dataset
train_data, test_data = random_split(train_dataset, [train_size, test_size])
print(f'train_data:{len(train_data)}, test_data:{len(test_data)}')

train_data:73257, test_data:0


In [13]:
# Step 4: Create a DataLoader for your dataset
train_loader = DataLoader(train_data, batch_size=216, shuffle=True)
test_loader = DataLoader(test_data, batch_size=216, shuffle=False)
print(len(train_loader), len(test_loader))

340 0


In [14]:
def vae_loss(reconstructed_x, x, mu, log_var):
    reconstruction_loss = F.binary_cross_entropy(reconstructed_x, x, reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return reconstruction_loss + kl_divergence

In [16]:
# Hyperparameters
image_channels = 3
h_dim = 1024
z_dim = 32
learning_rate = 1e-3

# 모델 및 옵티마이저 초기화
vae = VAE(image_channels=image_channels, h_dim=h_dim, z_dim=z_dim)
vae = vae.to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)
rec_loss = np.inf

# 학습 루프
for epoch in range(300):
    for batch in tqdm(train_loader):
        x, labels = batch['image'], batch['label']
        x = x.to(torch.float32).to(device)
        x = x.to(device)
        x_reconstructed, mu, log_var = vae(x)
        print('x:', x)
        print('x_reconstructed:', x_reconstructed)
        loss = vae_loss(x_reconstructed, x, mu, log_var)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if loss < rec_loss:
        torch.save(vae.state_dict(),'vae_best_model.pth')
    print(f'Epoch:{epoch} Reconstruction Loss:{loss}')

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
